In [10]:
import tensorflow as tf

In [16]:
dataset_path = 'Rice_Image_Dataset'

batch_size = 16
img_height = 200
img_width = 200

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 75000 files belonging to 5 classes.
Using 60000 files for training.


In [17]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 75000 files belonging to 5 classes.
Using 15000 files for validation.


In [22]:
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(train_ds.class_names), activation='softmax')
])

In [23]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 99, 99, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 48, 48, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 46, 46, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 23, 23, 128)      

In [25]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

Epoch 1/2
3750/3750 [==============================] - 2884s 769ms/step - loss: 0.3047 - accuracy: 0.9537 - val_loss: 0.1006 - val_accuracy: 0.9662
Epoch 2/2
3750/3750 [==============================] - 2732s 728ms/step - loss: 0.0966 - accuracy: 0.9694 - val_loss: 0.0693 - val_accuracy: 0.9770


In [27]:
cnn_loss, cnn_accuracy = model.evaluate(val_ds)
print(f"CNN Model Accuracy: {cnn_accuracy:.2f}")

938/938 [==============================] - 163s 174ms/step - loss: 0.0693 - accuracy: 0.9770
CNN Model Accuracy: 0.98


In [34]:
base_model = tf.keras.applications.EfficientNetB0(input_shape=(img_height, img_width, 3),
                                                   include_top=False,
                                                   weights='imagenet')
base_model.trainable = False  # Freeze the base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(train_ds.class_names), activation='softmax')
])


16705208/16705208 [==============================] - 4s 0us/step


In [35]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

Epoch 1/2
3750/3750 [==============================] - 2639s 700ms/step - loss: 0.0753 - accuracy: 0.9825 - val_loss: 0.0237 - val_accuracy: 0.9935
Epoch 2/2
3750/3750 [==============================] - 15688s 4s/step - loss: 0.0253 - accuracy: 0.9925 - val_loss: 0.0160 - val_accuracy: 0.9954
